1.

In [1]:
from sklearn.metrics import (accuracy_score, f1_score)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.utils import resample
import tensorflow as tf
import numpy as np
from random import sample 
from copy import deepcopy
from random import sample, randint, randrange
import imgaug.augmenters as iaa
from contextlib import contextmanager
from timeit import default_timer
from sklearn.model_selection import cross_validate

In [2]:
@contextmanager
def elapsed_timer():
    start_time = default_timer()

    class _Timer():
      start = start_time
      end = default_timer()
      duration = end - start

    yield _Timer

    end_time = default_timer()
    _Timer.end = end_time
    _Timer.duration = end_time - start_time

In [3]:
(x_train_f, y_train_f), (x_test_f, y_test_f) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# to jest do usuniecia
#x_train_f = x_train_f[:5000]
#y_train_f = y_train_f[:5000]
#
x_train = x_train_f.reshape((x_train_f.shape[0],-1))
x_test = x_test_f.reshape((x_test_f.shape[0],-1))
y_train = y_train_f.reshape((y_train_f.shape[0],))
y_test = y_test_f

scaler = StandardScaler()

scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

pca = PCA(n_components=50)
pca.fit(x_train_scaled)

x_train = pca.transform(x_train_scaled)
x_test = pca.transform(x_test_scaled)

X = np.concatenate([x_train, x_test])
Y = np.concatenate([y_train, y_test])

classes_count = 10

2.

Pełne dane

In [ ]:
def get_full_prediction(x_tr, y_tr, x_te):
    clf = SVC(probability=True)
    clf.fit(x_tr, y_tr)
    y_pred = clf.predict(x_te)
    pred = clf.predict_proba(x_te)
    return (y_pred, pred)

In [ ]:
scores = cross_validate(clf, svd_x_train, y_train, cv=5, scoring=('accuracy', 'neg_log_loss'))

In [ ]:
predictions = get_full_prediction(x_train, y_train, x_test)

In [ ]:
(y_pred, _) = predictions

In [ ]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='weighted'))

Niepełne dane i cechy

In [31]:
def average_pred(predictions):
    m = len(predictions[0][0])
    all_results = [[0 for x in range(classes_count)] for y in range(m)] 
    results = [0] * m
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

def majority_pred(predictions):
    m = len(predictions[0][0])
    results = [0] * m
    for i in range(m):
        all_results = [0 for x in range(classes_count)] 
        for (pred, _) in predictions:
            all_results[pred[i]] += 1
        results[i] = all_results.index(max(all_results))
        
    return results

def borda_pred(predictions):
    m = len(predictions[0][0])
    all_results = [[0 for x in range(classes_count)] for y in range(m)] 
    results = [0] * m
    
    def get_final_borda_points(predictions):
        return np.argsort(np.argsort(predictions)).tolist()

    for (_, pred_proba) in predictions:
        for i in range(m):
            pred_proba[i] = get_final_borda_points(pred_proba[i])
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

In [ ]:
class MinorClassifiers:
    def __init__(self, n_samples, n_features):
        self.n_samples = n_samples
        self.n_features = n_features
        self.classifiers = []
        self.predictions = []
        self.cut_features = []
    
    def get_params(self, deep = False):
        return {
            'n_samples': self.n_samples,
            'n_features': self.n_features,   
        }
    
    def predict(self, X):
        for i in range(self.classifiers):
            classifier = self.classifiers[i]
            x_test = X[:,self.cut_features[i]]
            y_pred = classifier.predict(x_test)
            pred = classifier.predict_proba(x_test)
            self.predictions.append((y_pred, pred))
            
        majority_pred(self.predictions)
        
    def fit(self, X, Y):
        feature_list = [n for n in range(50)]
        samples_all = X.shape[0]
        features_all = X.shape[1]

        for samples in self.n_samples:
            for features in self.n_features:
                f = sample(feature_list, int(features_all * features))
                self.cut_features.append(f)
                x_train_f = X[:,f]
                
                x_train_s, y_train_s = resample(x_train_f, Y, n_samples=int(samples * samples_all), replace=False, random_state=0)

                svm_clf = SVC(probability=True)
                svm_clf.fit(x_train_s, y_train_s)
                
                self.classifiers.append(svm_clf)

In [ ]:
n_samples = [0.1, 0.25, 0.35, 0.5]
n_features = [0.25, 0.5, 0.75]

In [ ]:
minor = MinorClassifiers(n_samples, n_features)

In [ ]:
scores = cross_validate(minor, X, Y, cv=5, scoring=('accuracy', 'neg_log_loss'))

In [ ]:
def get_minor_predictions(x_tr, y_tr, x_te, samples_perc, features_perc):
    feature_list = [n for n in range(50)]
    predictions = []
    samples_all = x_tr.shape[0]
    features_all = x_tr.shape[1]
    
    for samples in samples_perc:
        for features in features_perc:
            f = sample(feature_list, int(features_all * features))
            x_train_f = x_tr[:,f]
            x_test_f = x_te[:,f]
            
            x_train_s, y_train_s = resample(x_train_f, y_tr, n_samples=int(samples * samples_all), replace=False, random_state=0)
            
            svm_clf = SVC(probability=True)
            svm_clf.fit(x_train_s, y_train_s)
            
            y_pred = svm_clf.predict(x_test_f)
            pred = svm_clf.predict_proba(x_test_f)
            
            predictions.append((y_pred, pred))
            
    return predictions

In [ ]:
pred = get_minor_predictions(x_train, y_train, x_test, n_samples, n_features)

In [ ]:
print(accuracy_score(y_test, majority_pred(pred)))
print(f1_score(y_test, majority_pred(pred), average='weighted'))

In [ ]:
print(accuracy_score(y_test, average_pred(pred)))
print(f1_score(y_test, average_pred(pred), average='weighted'))

In [ ]:
print(accuracy_score(y_test, borda_pred(pred)))
print(f1_score(y_test, borda_pred(pred), average='weighted'))

3a.

In [ ]:
clf_100iter = SVC(probability=True, max_iter=100)

In [ ]:
with elapsed_timer() as t:
    clf_100iter.fit(x_train, y_train)

print("duration: " + str(t.duration))

In [ ]:
clf_7iter.fit(x_train, y_train)
y_pred_7ter = clf_7iter.predict(x_test)
pred_7iter = clf_7iter.predict_proba(x_test)

In [ ]:
pred_7iter

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_7iter = accuracy_score(y_test, y_pred_7ter)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy_7iter

In [ ]:
accuracy

4. 

In [7]:
def make_noise_data(data, percent):
    m = data.shape[0]
    n = data.shape[1]
    new_data = deepcopy(data)
    for i in range(m):
        for j in range(n):
            rand = randrange(-1, 2, 2) #random integer from {-1, 1}
            new_data[i][j] = (1 + rand * percent) * data[i][j]
    return new_data

In [ ]:
def make_noise_label(labels, percent):
    labels_with_noise = deepcopy(labels)
    arr_size = labels.shape[0]
    indexes = [n for n in range(arr_size)]
    indexes_to_change = sample(indexes, int(arr_size * percent))
    
    for i in indexes_to_change:
        old_val = labels[i]
        new_val = randint(0, classes_count)
        while old_val == new_val:
            new_val = randint(0, classes_count)
        labels_with_noise[i] = new_val
        
    return labels_with_noise

In [ ]:
new_y_train = make_noise_label(y_train, 0.2)

In [ ]:
pred = get_minor_predictions(x_train, new_y_train, x_test, n_samples, n_features)

In [ ]:
print(accuracy_score(y_test, majority_pred(pred)))
print(f1_score(y_test, majority_pred(pred), average='weighted'))

In [11]:
new_x_train = make_noise_data(x_train, 0.2)

In [ ]:
pred = get_minor_predictions(new_x_train, y_train, x_test, n_samples, n_features)

In [ ]:
print(accuracy_score(y_test, majority_pred(pred)))
print(f1_score(y_test, majority_pred(pred), average='weighted'))